In [13]:
import torch
import torch.optim as optim

from NGCF import NGCF

from utility.helper import *

In [14]:
from utility.batch_test import *


usage: ipykernel_launcher.py [-h] [--weights_path [WEIGHTS_PATH]] [--data_path [DATA_PATH]] [--proj_path [PROJ_PATH]]
                             [--dataset [DATASET]] [--pretrain PRETRAIN] [--verbose VERBOSE] [--epoch EPOCH] [--embed_size EMBED_SIZE]
                             [--layer_size [LAYER_SIZE]] [--batch_size BATCH_SIZE] [--regs [REGS]] [--lr LR] [--model_type [MODEL_TYPE]]
                             [--adj_type [ADJ_TYPE]] [--gpu_id GPU_ID] [--node_dropout_flag NODE_DROPOUT_FLAG]
                             [--node_dropout [NODE_DROPOUT]] [--mess_dropout [MESS_DROPOUT]] [--Ks [KS]] [--save_flag SAVE_FLAG]
                             [--test_flag [TEST_FLAG]] [--report REPORT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/rok/.local/share/jupyter/runtime/kernel-935a6840-6506-46c2-9962-9f650370436e.json


SystemExit: 2

In [25]:
import warnings
warnings.filterwarnings('ignore')
from time import time

* load_data.py 역할

In [63]:
data_path = '../Data/' #절대경로
dataset = 'amazon-book' #절대경로
batch_size = 1024 #절대경로
data_generator = Data(path=data_path + dataset, batch_size=batch_size)

n_users=52643, n_items=91599
n_interactions=2984108
n_train=2380730, n_test=603378, sparsity=0.00062


* load_data >>> Data클래스 >>> data_generator 인스턴스의 변수

In [22]:
USR_NUM, ITEM_NUM = data_generator.n_users, data_generator.n_items
N_TRAIN, N_TEST = data_generator.n_train, data_generator.n_test
BATCH_SIZE = batch_size

In [23]:
USR_NUM, ITEM_NUM, N_TRAIN, N_TEST, BATCH_SIZE

(52643, 91599, 2380730, 603378, 1024)

In [32]:
print(f' data_generator.exist_users[:10] = {data_generator.exist_users[:10]}')

 data_generator.exist_users[:10] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [34]:
data_generator.print_statistics()

n_users=52643, n_items=91599
n_interactions=2984108
n_train=2380730, n_test=603378, sparsity=0.00062


In [35]:
#scipy의 sparse 함수
data_generator.R = sp.dok_matrix((data_generator.n_users, data_generator.n_items), dtype=np.float32)

In [64]:
data_generator.R

<52643x91599 sparse matrix of type '<class 'numpy.float32'>'
	with 2380730 stored elements in Dictionary Of Keys format>

In [45]:
#train set에서 1234번 user의 구매아이템
data_generator.train_items[1234][:3]

[24644, 39134, 27389]

* 다시 main.py

In [49]:
gpu_id=0
device = torch.device('cuda:' + str(gpu_id))
device

device(type='cuda', index=0)

* load_data.py >>> get_adj_mat 실행  >>> create_adj_mat 함수 >>> adj_mat, norm_mat, mean_mat 생성

In [65]:
#adj 생성
adj_mat = sp.dok_matrix((data_generator.n_users + data_generator.n_items, data_generator.n_users + data_generator.n_items), dtype=np.float32) # (사용자수+아이템수) x (사용자수+아이템수) 깡통 행렬 생성
adj_mat = adj_mat.tolil() # matrix를 list속 list형태로 변환
tmp_R = data_generator.R.tolil() # (사용자수)x(아이템수) tolil 변환


In [73]:
tmp_R[0]

<1x91599 sparse matrix of type '<class 'numpy.float32'>'
	with 53 stored elements in List of Lists format>

In [74]:
adj_mat[:data_generator.n_users, data_generator.n_users:] = tmp_R

In [75]:
adj_mat[data_generator.n_users:, :data_generator.n_users] = tmp_R.T

In [86]:
#인접행렬내 0번사용자의 벡터
adj_mat[0]

<1x144242 sparse matrix of type '<class 'numpy.float32'>'
	with 53 stored elements in Dictionary Of Keys format>

In [78]:
adj_mat = adj_mat.todok() # matrix를 딕셔너리의 형태로 변환

In [82]:
type(adj_mat)

scipy.sparse._dok.dok_matrix

In [50]:
plain_adj, norm_adj, mean_adj = data_generator.get_adj_mat()

already create adjacency matrix (144242, 144242) 189.67632484436035
generate single-normalized adjacency matrix.
generate single-normalized adjacency matrix.
already normalize adjacency matrix 0.041094303131103516


In [87]:
rowsum = np.array(adj_mat.sum(1))

In [89]:
rowsum.shape

(144242, 1)

In [91]:
rowsum[1]

array([328.], dtype=float32)

In [92]:
d_inv = np.power(rowsum, -1).flatten()

In [95]:
d_mat_inv = sp.diags(d_inv)

In [100]:
d_mat_inv

<144242x144242 sparse matrix of type '<class 'numpy.float32'>'
	with 144242 stored elements (1 diagonals) in DIAgonal format>

In [102]:
adj_mat

<144242x144242 sparse matrix of type '<class 'numpy.float32'>'
	with 4761460 stored elements in Dictionary Of Keys format>

In [104]:
norm_adj = d_mat_inv.dot(adj_mat)

In [112]:
adj_mat.sum(0)

matrix([[ 53., 328.,  47., ...,   1.,   1.,   1.]], dtype=float32)

In [111]:
norm_adj.sum(0)

matrix([[ 1.966299  , 13.315645  ,  1.538288  , ...,  0.03571429,
          0.0625    ,  0.0625    ]], dtype=float32)

In [115]:
mean_adj.sum(0)

matrix([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:




args.node_dropout = eval(args.node_dropout)
args.mess_dropout = eval(args.mess_dropout)

model = NGCF(data_generator.n_users,
             data_generator.n_items,
             norm_adj,
             args).to(args.device)

t0 = time()
"""
*********************************************************
Train.
"""
cur_best_pre_0, stopping_step = 0, 0
optimizer = optim.Adam(model.parameters(), lr=args.lr)

loss_loger, pre_loger, rec_loger, ndcg_loger, hit_loger = [], [], [], [], []